In [2]:
#IMPORTO LE LIBRERIE 
import pandas as pd

In [3]:
#LETTURA FILE CSV capire come rendere globale questa url
url_AIDS_Classification = r'C:\Users\Luca\Desktop\Develhope\Project\AIDS_VirusInfectionPrediction\data\AIDS_Classification_5000.csv'
df = pd.read_csv(url_AIDS_Classification)
print(df.head())

   time  trt  age      wtkg  hemo  homo  drugs  karnof  oprior  z30  ...  \
0   988    2   42  71.76131     0     0      0     100       0    1  ...   
1   413    3   30  72.67629     0     1      0     100       0    1  ...   
2  1119    0   37  73.75514     0     1      0      90       0    1  ...   
3   680    3   34  70.03769     0     0      0     100       0    0  ...   
4  1043    3   35  63.02845     0     0      0     100       0    1  ...   

   str2  strat  symptom  treat  offtrt  cd40  cd420  cd80  cd820  infected  
0     1      3        0      0       0   290    408  1870   1671         0  
1     0      1        0      0       0   506    446  1372    523         0  
2     1      3        0      0       0   499    542   609   1474         0  
3     0      1        0      1       0   313    478   780    987         0  
4     0      1        0      1       0   337    761   698   1735         0  

[5 rows x 23 columns]
